### establish a percent change metric
rolling forward

In [2]:
!pip3 install matplotlib
!pip3 install altair

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [40]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [4]:
!mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

grep: ./lob_caps: Is a directory
usage: mv [-f | -i | -n] [-hv] source target
       mv [-f | -i | -n] [-v] source ... directory


In [5]:
#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  284997
start:  1660221600292.0  end:  1691444880101.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


In [6]:
def getSKEWByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getSKEWByDateAndType("MEANSHIFT") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

skewFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
skewFrame.sort_values(by=['timeStamp'], ascending=True)   #sorted by time into one time series
skewFrame.rename(columns={'timeStamp': 'time'}, inplace=True)
print("for new df: ", skewFrame.shape[0])
start = skewFrame["time"].min()
end = skewFrame["time"].max()
print("start: ", start, " end: ", end)
print(skewFrame.columns)

for new df:  139237
start:  1660221606227  end:  1691444966359
Index(['mp', 'time', 'mean', 'skew'], dtype='object')


In [7]:
# Merging those two data frames Will not take place based on a Shared key of time
# You'll need to look up the SKU value for every row based on approximation
# merged_df = pd.merge(capsFrame, skewFrame, on='time')

In [8]:
skewFrame.head(10)

,mp,time,mean,skew
0,17.66,1677651424290,5877.906080,-5860.246080
1,17.66,1677651427404,5906.691222,-5889.031222
2,17.65,1677651430373,5944.892609,-5927.242609
3,17.66,1677651436323,5950.952405,-5933.292405
4,17.66,1677651439323,5926.797221,-5909.137221
5,17.66,1677651442399,5909.721180,-5892.061180
6,17.66,1677651445381,5898.866475,-5881.206475
7,17.66,1677651448534,5899.116796,-5881.456796
8,17.66,1677651451564,5891.509682,-5873.849682
9,17.66,1677651454328,5891.185193,-5873.525193


### Apply skew and mean values from mean shift to the caps data frame FAIL

for the data frame capsFrame set the column of 'mean' equal to the data frame, skewFrame 'mean' where the 'time' column in both data frames is approximately equal to within one minute



In [9]:
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'mean'] = skewFrame['mean'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'skew'] = skewFrame['skew'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().lt(pd.Timedelta(minutes=2)).idxmax(), 'mean'] = skewFrame['mean'].values[0]
# 
# Could not get this operation to work try it again using a range of values once the precursor and surge are defined


In [10]:
capsFrame.head(50)

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74
2,2424788.47,10557236.44,492823.44,244815.55,1.672418e+12,10.78,10.74
3,2423662.55,10557235.85,492718.37,244815.55,1.672418e+12,10.78,10.74
4,2424625.42,10557236.44,492808.51,244815.55,1.672418e+12,10.78,10.74
5,2427126.58,10557249.44,493040.74,244815.55,1.672418e+12,10.78,10.74
6,2426385.79,10554446.08,492971.86,244556.04,1.672418e+12,10.78,10.74
7,2419624.39,10555247.16,492344.04,244630.14,1.672418e+12,10.78,10.74
8,2427090.01,10557088.36,493037.44,244800.94,1.672418e+12,10.78,10.74
9,2427125.46,10557086.67,493040.74,244800.94,1.672418e+12,10.78,10.74


In [11]:


# Load your time series data into a pandas dataframe
caps_df = pd.read_csv('Fri Apr 07 2023 12:26:51 GMT-0700 (Pacific Daylight Time)-CAPS.csv')


lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

926 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change'], dtype='object')


In [12]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

-3.996e-05

In [13]:
#reset mean chang ebery 24 hrs

In [14]:
# identify units of 10 rows where the percent change is greater or less than the threshold

threshold = meanChange
surges = []
precursors = []

for i in range(0,len(caps_df),10):

    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        
        surges.append({'time': caps_df.iloc[i]['time'],
                       'MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'bucket':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                        'bucket':'precursor',
                        'buyCap':caps_df.iloc[i]['bc'], 
                        'askCap':caps_df.iloc[i]['ac'],
                        'totalBidVol':caps_df.iloc[i]['tbv'],
                        'totalAskVol':caps_df.iloc[i]['tav']})  

In [15]:
for item in surges:
    print(item)


{'time': 1680895735340.0, 'MP': 17.59, 'change': 0.0, 'bucket': 'surge'}
{'time': 1680895765299.0, 'MP': 17.59, 'change': 0.0, 'bucket': 'surge'}
{'time': 1680895795332.0, 'MP': 17.59, 'change': 0.0, 'bucket': 'surge'}
{'time': 1680895826633.0, 'MP': 17.59, 'change': 0.00039795338260384394, 'bucket': 'surge'}
{'time': 1680895863002.0, 'MP': 17.6, 'change': 0.0001705514496873617, 'bucket': 'surge'}
{'time': 1680895935966.0, 'MP': 17.6, 'change': 0.00011370096645824113, 'bucket': 'surge'}
{'time': 1680895992486.0, 'MP': 17.6, 'change': 0.00017045454545452365, 'bucket': 'surge'}
{'time': 1680896047452.0, 'MP': 17.61, 'change': 0.00039772727272722184, 'bucket': 'surge'}
{'time': 1680896203843.0, 'MP': 17.6, 'change': 5.685044654391014e-05, 'bucket': 'surge'}
{'time': 1680896290548.0, 'MP': 17.62, 'change': 0.0007384428851511471, 'bucket': 'surge'}
{'time': 1680896347965.0, 'MP': 17.63, 'change': 0.0003972758229283446, 'bucket': 'surge'}
{'time': 1680896456634.0, 'MP': 17.63, 'change': 0.00

In [16]:
for item in precursors:
    print(item)

{'time': 1680895615420.0, 'MP': 17.61, 'change': nan, 'bucket': 'precursor', 'buyCap': 3537895.45, 'askCap': 32564599.64, 'totalBidVol': 417439.52, 'totalAskVol': 328777.66}
{'time': 1680895645356.0, 'MP': 17.61, 'change': -0.0003975014196478499, 'bucket': 'precursor', 'buyCap': 3522640.95, 'askCap': 32577586.24, 'totalBidVol': 416576.54, 'totalAskVol': 329515.14}
{'time': 1680895675317.0, 'MP': 17.6, 'change': -0.00017035775127764995, 'bucket': 'precursor', 'buyCap': 3519748.95, 'askCap': 32551526.13, 'totalBidVol': 416417.82, 'totalAskVol': 328048.27}
{'time': 1680895705306.0, 'MP': 17.59, 'change': -0.0005681818181818565, 'bucket': 'precursor', 'buyCap': 3512363.25, 'askCap': 32549903.68, 'totalBidVol': 416014.34, 'totalAskVol': 327997.32}
{'time': 1680895900177.0, 'MP': 17.6, 'change': -0.00011363636363637131, 'bucket': 'precursor', 'buyCap': 3510978.75, 'askCap': 32528767.96, 'totalBidVol': 415987.69, 'totalAskVol': 326772.43}
{'time': 1680896117048.0, 'MP': 17.61, 'change': -5.67

In [17]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)


In [18]:
for index, row in sequence_df.iterrows():
    print(row['bucket'])


precursor
precursor
precursor
precursor
surge
surge
surge
surge
surge
precursor
surge
surge
surge
precursor
surge
surge
surge
surge
surge
surge
surge
precursor
precursor
surge
precursor
precursor
precursor
precursor
precursor
precursor
precursor
surge
surge
surge
precursor
precursor
surge
surge
precursor
precursor
surge
surge
surge
precursor
surge
surge
surge
precursor
precursor
precursor
precursor
precursor
precursor
precursor
precursor
precursor
surge
surge
surge
precursor
surge
surge
surge
precursor
surge
surge
surge
surge
surge
surge
surge
precursor
precursor
surge
precursor
surge
precursor
surge
surge
surge
precursor
precursor
precursor
surge
surge
precursor
precursor
surge
precursor
precursor
precursor
surge
precursor


In [19]:
sequence_df.head(45)

,time,MP,change,bucket,buyCap,askCap,totalBidVol,totalAskVol
0,1.680896e+12,17.61,NaN,precursor,3537895.45,32564599.64,417439.52,328777.66
1,1.680896e+12,17.61,-3.975014e-04,precursor,3522640.95,32577586.24,416576.54,329515.14
2,1.680896e+12,17.60,-1.703578e-04,precursor,3519748.95,32551526.13,416417.82,328048.27
3,1.680896e+12,17.59,-5.681818e-04,precursor,3512363.25,32549903.68,416014.34,327997.32
0,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN
1,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN
2,1.680896e+12,17.59,0.000000e+00,surge,NaN,NaN,NaN,NaN
3,1.680896e+12,17.59,3.979534e-04,surge,NaN,NaN,NaN,NaN
4,1.680896e+12,17.60,1.705514e-04,surge,NaN,NaN,NaN,NaN
4,1.680896e+12,17.60,-1.136364e-04,precursor,3510978.75,32528767.96,415987.69,326772.43


In [20]:

line = alt.Chart(sequence_df).mark_line(color='green').encode(
    x='time',
    y='MP'
)

bar = alt.Chart(sequence_df).mark_bar().encode(
    x='time',
    y='MP',
    color='bucket'
)

chart = (line + bar).properties(width=600, height=500)
chart.title = 'Sequential order of precursor and surges for April 7th 2023'

subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
chart

alt.LayerChart(...)

In [21]:
sequence_df.columns

Index(['time', 'MP', 'change', 'bucket', 'buyCap', 'askCap', 'totalBidVol',
       'totalAskVol'],
      dtype='object')

### Grouping together bucketed surge items

In [44]:

sequence_df['group'] = (sequence_df['bucket'] != sequence_df['bucket'].shift(1)).cumsum()
sequence_df['length'] = sequence_df.groupby(['bucket', 'group'])['group'].transform('count')
sequence_df['sum'] = sequence_df.groupby(['bucket', 'group'])['change'].transform('sum')
sequence_df['identifier'] = sequence_df.groupby(['bucket', 'group'])['time'].transform('min')
sequence_df['end_time'] = sequence_df.groupby(['bucket', 'group'])['time'].transform('max')
sequence_df['type'] = sequence_df['bucket']
sequence_df['buyCapSum'] = sequence_df.groupby(['bucket', 'group'])['buyCap'].transform('sum')
sequence_df['askCapSum'] = sequence_df.groupby(['bucket', 'group'])['askCap'].transform('sum')
print(sequence_df)

            time     MP    change     bucket      buyCap       askCap  \
0   1.680896e+12  17.61       NaN  precursor  3537895.45  32564599.64   
1   1.680896e+12  17.61 -0.000398  precursor  3522640.95  32577586.24   
2   1.680896e+12  17.60 -0.000170  precursor  3519748.95  32551526.13   
3   1.680896e+12  17.59 -0.000568  precursor  3512363.25  32549903.68   
0   1.680896e+12  17.59  0.000000      surge         NaN          NaN   
..           ...    ...       ...        ...         ...          ...   
40  1.680975e+12  17.64 -0.001700  precursor  3495196.49  32556829.73   
41  1.680977e+12  17.57 -0.001588  precursor  3515728.64  32554076.15   
42  1.680979e+12  17.57 -0.003353  precursor  3531971.49  32560898.32   
48  1.680981e+12  17.53  0.000856      surge         NaN          NaN   
43  1.680984e+12  17.53 -0.000474  precursor  3514697.92  32545381.14   

    totalBidVol  totalAskVol  group  length       sum    identifier  \
0     417439.52    328777.66      1       4 -0.00113

In [34]:
# sequence_df = sequence_df.drop('next_value', axis=1)
# sequence_df.loc[sequence_df['bucket'] == 'surge', 'surge_length'] =  sequence_df['length']
# sequence_df.drop('length', axis=1, inplace=True)
# df = df.loc[:,~df.columns.duplicated()]


In [45]:
print(sequence_df)

            time     MP    change     bucket      buyCap       askCap  \
0   1.680896e+12  17.61       NaN  precursor  3537895.45  32564599.64   
1   1.680896e+12  17.61 -0.000398  precursor  3522640.95  32577586.24   
2   1.680896e+12  17.60 -0.000170  precursor  3519748.95  32551526.13   
3   1.680896e+12  17.59 -0.000568  precursor  3512363.25  32549903.68   
0   1.680896e+12  17.59  0.000000      surge         NaN          NaN   
..           ...    ...       ...        ...         ...          ...   
40  1.680975e+12  17.64 -0.001700  precursor  3495196.49  32556829.73   
41  1.680977e+12  17.57 -0.001588  precursor  3515728.64  32554076.15   
42  1.680979e+12  17.57 -0.003353  precursor  3531971.49  32560898.32   
48  1.680981e+12  17.53  0.000856      surge         NaN          NaN   
43  1.680984e+12  17.53 -0.000474  precursor  3514697.92  32545381.14   

    totalBidVol  totalAskVol  group  length       sum    identifier  \
0     417439.52    328777.66      1       4 -0.00113

### Is pandas vector is ation two iterate sequences data frame summarize the real contents then extract one representative row for that type of sequence

Be sure. Of attributes has been moved from the caps data frame. Be prepared to have a start and end time stamp. Match the SKU and mean from mean shift given the range of start and end in caps merge the data sets from the time field.

In [47]:
unique_df = sequence_df.groupby('identifier').first().reset_index()
unique_df.loc[unique_df['bucket'] == 'surge', 'surge_length'] = unique_df['length']
unique_df.loc[unique_df['bucket'] == 'surge', 'length'] = 0

print(unique_df)  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']


      identifier          time     MP        change     bucket      buyCap  \
0   1.680896e+12  1.680896e+12  17.61 -3.975014e-04  precursor  3537895.45   
1   1.680896e+12  1.680896e+12  17.59  0.000000e+00      surge         NaN   
2   1.680896e+12  1.680896e+12  17.60 -1.136364e-04  precursor  3510978.75   
3   1.680896e+12  1.680896e+12  17.60  1.137010e-04      surge         NaN   
4   1.680896e+12  1.680896e+12  17.61 -5.678592e-05  precursor  3521737.66   
5   1.680896e+12  1.680896e+12  17.60  5.685045e-05      surge         NaN   
6   1.680897e+12  1.680897e+12  17.64 -3.968254e-04  precursor  3528630.16   
7   1.680897e+12  1.680897e+12  17.62  1.702933e-04      surge         NaN   
8   1.680898e+12  1.680898e+12  17.64 -2.267573e-04  precursor  3516764.02   
9   1.680899e+12  1.680899e+12  17.57  3.882788e-07      surge         NaN   
10  1.680901e+12  1.680901e+12  17.59 -1.987636e-03  precursor  3514346.80   
11  1.680902e+12  1.680902e+12  17.57  9.674936e-04      surge  

In [25]:
unique_df = unique_df.groupby(unique_df.index // 2).agg({'identifier': 'first', 'time': 'first', 'MP': 'first', \
    'change': 'first', 'bucket': 'first', 'group': 'first', 'length': lambda x: x.iloc[0] if x.index[0] % 2 == 0 else x.sum(),\
         'surge_length': lambda x: x.iloc[0] if x.index[0] % 2 == 1 else x.sum()})


KeyError: "Column(s) ['surge_length'] do not exist"